In [ ]:
!pip install -U "transformers" "huggingface_hub" "tokenizers"
!pip install Levenshtein
!pip install "protobuf==3.20.3" --force-reinstall
!pip install llm2vec


In [ ]:
import json
import pandas as pd
import numpy as np
import json
import pandas as pd
import numpy as np
import os, math, pandas as pd, numpy as np
from dataclasses import dataclass
from typing import Dict, List, Union, Optional
import torch
import Levenshtein
from typing import List, Sequence, Tuple
from transformers import (AutoTokenizer, AutoConfig, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, DataCollatorWithPadding)
from sklearn.metrics import accuracy_score, f1_score
from scipy.stats import spearmanr, pearsonr
import editdistance
from peft import LoraConfig, get_peft_model, TaskType
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, AutoConfig
from tqdm import tqdm
import os
import random
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftModel
from collections import deque
from tqdm import tqdm
import os
import torch
import torch.optim as optim
import numpy as np
from transformers import get_scheduler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.cuda.amp import autocast, GradScaler
import kagglehub
import random
from scipy.stats import spearmanr

from types import SimpleNamespace

In [ ]:
TASK_TYPE = "pair_cls"
MAX_LENGTH = 256
BATCH_SIZE = 4
epochs = 5
LR = 2e-5
warmup_ratio = 0.06
save_dir = 'ckpt1'
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

###
blending_model_special_token = "_" # teacher
base_model_special_token = "##" # student

In [ ]:


class TextPairRaw(Dataset):
    def __init__(self, df: pd.DataFrame, task: str):
        self.task = task
        if task == "single_cls":
            self.samples = [(t, None, int(y)) for t, y in zip(df["text"].astype(str), df["label"].astype(int))]
        elif task == "pair_cls":
            self.samples = [(a, b) for a,b in zip(df["premise"].astype(str),
                                                            df["hypothesis"].astype(str))]
        else:  # pair_reg
            self.samples = [(a, b) for a,b in zip(df["sentence1"].astype(str),
                                                              df["sentence2"].astype(str))]
    def __len__(self): return len(self.samples)
    def __getitem__(self, idx): return self.samples[idx] 
from typing import List, Tuple, Optional



class DualTokenizerCollate:
    def __init__(self, tok_student, tok_teacher, task: str, max_len: int):
        self.ts = tok_student
        self.tt = tok_teacher
        self.task = task
        self.max_len = max_len

    def __call__(self, batch: List[Tuple[str, Optional[str], float]]):
        s1s, s2s = zip(*batch)

        if self.task == "single_cls":
            s_enc = self.ts(list(s1s), max_length=self.max_len, truncation=True,
                            padding=True, return_tensors="pt",
                            return_special_tokens_mask=True)
            t_enc = self.tt(list(s1s), max_length=self.max_len, truncation=True,
                            padding=True, return_tensors="pt",
                            return_special_tokens_mask=True)

            out = {
                "input_ids_stu": s_enc["input_ids"],
                "attention_mask_stu": s_enc["attention_mask"],
                "special_tokens_mask_stu": s_enc["special_tokens_mask"],
                "input_ids_tea": t_enc["input_ids"],
                "attention_mask_tea": t_enc["attention_mask"],
                "special_tokens_mask_tea": t_enc["special_tokens_mask"],
                "labels": torch.tensor(ys, dtype=torch.long),
            }
            if "token_type_ids" in s_enc:
                out["token_type_ids_stu"] = s_enc["token_type_ids"]
            if "token_type_ids" in t_enc:
                out["token_type_ids_tea"] = t_enc["token_type_ids"]
            return out

        # ------- pair (bi-encoder) -------
        s1_enc = self.ts(list(s1s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)
        s2_enc = self.ts(list(s2s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)

        t1_enc = self.tt(list(s1s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)
        t2_enc = self.tt(list(s2s), max_length=self.max_len, truncation=True,
                         padding=True, return_tensors="pt",
                         return_special_tokens_mask=True)

        out = {
            # student
            "input_ids1_stu": s1_enc["input_ids"],
            "attention_mask1_stu": s1_enc["attention_mask"],
            "special_tokens_mask1_stu": s1_enc["special_tokens_mask"],
            "input_ids2_stu": s2_enc["input_ids"],
            "attention_mask2_stu": s2_enc["attention_mask"],
            "special_tokens_mask2_stu": s2_enc["special_tokens_mask"],
            # teacher
            "input_ids1_tea": t1_enc["input_ids"],
            "attention_mask1_tea": t1_enc["attention_mask"],
            "special_tokens_mask1_tea": t1_enc["special_tokens_mask"],
            "input_ids2_tea": t2_enc["input_ids"],
            "attention_mask2_tea": t2_enc["attention_mask"],
            "special_tokens_mask2_tea": t2_enc["special_tokens_mask"],
        }
        # chỉ thêm token_type_ids nếu tồn tại
        if "token_type_ids" in s1_enc:
            out["token_type_ids1_stu"] = s1_enc["token_type_ids"]
        if "token_type_ids" in s2_enc:
            out["token_type_ids2_stu"] = s2_enc["token_type_ids"]
        if "token_type_ids" in t1_enc:
            out["token_type_ids1_tea"] = t1_enc["token_type_ids"]
        if "token_type_ids" in t2_enc:
            out["token_type_ids2_tea"] = t2_enc["token_type_ids"]

        return out



print("Done Prepare Dataset")

In [ ]:
def info_nce(q, k, temperature=0.07, neg_valid_mask=None):
    q = F.normalize(q, dim=-1)
    k = F.normalize(k, dim=-1)

    logits = torch.matmul(q, k.T) / temperature
    labels = torch.arange(q.size(0), device=q.device)
    loss_inbatch = F.cross_entropy(logits, labels) 
    return loss_inbatch, logits

In [ ]:
data_train = '/kaggle/input/multitask-data/merged_9_data_3k_each_ver2.csv'
df = pd.read_csv(data_train)
df.head(3)

In [ ]:
df["premise"] = df["text"]
df["hypothesis"] = df["text"]
cols = ["premise", "hypothesis"]  # hoặc ["premise","hypothesis","label"] nếu cần placeholder
df_out = df[cols].copy()

In [ ]:
student_name = "huawei-noah/TinyBERT_General_4L_312D"
tok_student = AutoTokenizer.from_pretrained(student_name)
model_student = AutoModel.from_pretrained(
    student_name,
    output_hidden_states=True
)

tok_teacher = AutoTokenizer.from_pretrained("BAAI/bge-m3")
model_teacher = AutoModel.from_pretrained(
    "BAAI/bge-m3",
    output_hidden_states=True
)

In [ ]:

train_ds = TextPairRaw(df_out, TASK_TYPE)

collate = DualTokenizerCollate(tok_student, tok_teacher, TASK_TYPE, MAX_LENGTH)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=collate, pin_memory=True, num_workers=2, persistent_workers=True)
print("Done Prepare Dataloader")

In [ ]:
import Levenshtein
from typing import List, Sequence, Tuple
import editdistance
specTok_mapper = {}

# Map thủ công theo loại token
if tok_student.cls_token and tok_teacher.bos_token:
    specTok_mapper[tok_student.cls_token] = tok_teacher.bos_token
if tok_student.sep_token and tok_teacher.eos_token:
    specTok_mapper[tok_student.sep_token] = tok_teacher.eos_token
if tok_student.pad_token and tok_teacher.pad_token:
    specTok_mapper[tok_student.pad_token] = tok_teacher.pad_token
if tok_student.unk_token and tok_teacher.unk_token:
    specTok_mapper[tok_student.unk_token] = tok_teacher.unk_token
if tok_student.mask_token and tok_teacher.mask_token:
    specTok_mapper[tok_student.mask_token] = tok_teacher.mask_token



def dist_fn(a, b):
    """Calculate editdistance between two tokens, a is from blending model, b is from base model."""
    if a in specTok_mapper and b in specTok_mapper.values():
        return 0.0
    if b in specTok_mapper and a in specTok_mapper.values():
        return 0.0
    aa = a.replace(blending_model_special_token, "")
    bb = b.replace(base_model_special_token, "")
    aa = aa.replace(" ", "")
    bb = bb.replace(" ", "")
    dist = editdistance.eval(aa, bb) 
    if len(aa)==len(bb)==0:
        return 0.0
    dist = dist / (len(aa)+len(bb))
    return dist
def cost_fn(a, b):
    """cost function for sequence alignment"""
    if a in specTok_mapper and b in specTok_mapper.values():
        return 0.0
    if b in specTok_mapper and a in specTok_mapper.values():
        return 0.0
    aa = a.replace(blending_model_special_token, "")
    bb = b.replace(base_model_special_token, "")
    aa = aa.replace(" ", "")
    bb = bb.replace(" ", "")
    dist = editdistance.eval(aa, bb)
    return dist

def dtw(series_1, series_2, series1_factor=None, series2_factor=None, norm_func=np.linalg.norm): # dist_fn
    matrix = np.zeros((len(series_1) + 1, len(series_2) + 1)) # [max_len blending] x [max_len student] , tất cả là 0
    matrix[0, :] = np.inf # toàn bộ hàng ngang đầu là inf
    matrix[:, 0] = np.inf # toàn bộ hàng dọc đầu là inf
    matrix[0, 0] = 0 # vị trí bắt đầu là 0
    if series1_factor is not None and series2_factor is not None:
        for i, (vec1, fc1) in enumerate(zip(series_1, series1_factor)): 
            for j, (vec2, fc2) in enumerate(zip(series_2, series2_factor)):
                cost = norm_func(vec1, vec2) * fc1 * fc2 
                matrix[i + 1, j + 1] = cost + min(
                    matrix[i, j + 1], matrix[i + 1, j], matrix[i, j]
                )
    else:
        for i, vec1 in enumerate(series_1): 
            for j, vec2 in enumerate(series_2):
                cost = norm_func(vec1, vec2) 
                matrix[i + 1, j + 1] = cost + min(
                    matrix[i, j + 1], matrix[i + 1, j], matrix[i, j]
                )

    matrix = matrix[1:, 1:] # giảm 1 cho 2 chiều
    i = matrix.shape[0] - 1 
    j = matrix.shape[1] - 1
    # đến đây là xong ma trận 
    matches = []
    mappings_series_1 = [list() for v in range(matrix.shape[0])]
    mappings_series_2 = [list() for v in range(matrix.shape[1])]
    while i > 0 or j > 0:
        matches.append((i, j))
        mappings_series_1[i].append(j)
        mappings_series_2[j].append(i)
        option_diag = matrix[i - 1, j - 1] if i > 0 and j > 0 else np.inf
        option_up = matrix[i - 1, j] if i > 0 else np.inf
        option_left = matrix[i, j - 1] if j > 0 else np.inf
        move = np.argmin([option_diag, option_up, option_left])
        if move == 0:
            i -= 1
            j -= 1
        elif move == 1:
            i -= 1
        else:
            j -= 1
    matches.append((0, 0))
    mappings_series_1[0].append(0)
    mappings_series_2[0].append(0)
    matches.reverse()
    for mp in mappings_series_1:
        mp.reverse()
    for mp in mappings_series_2:
        mp.reverse()

    return matches, matrix[-1, -1], mappings_series_1, mappings_series_2, matrix


import torch
import numpy as np
from typing import List, Sequence, Tuple

def _normalize_token(t: str, special_marker: str) -> str:
    # Bỏ marker (e.g., "##", "▁", "Ġ"), bỏ space, lower-case
    if special_marker:
        t = t.replace(special_marker, "")
    t = t.replace(" ", "").lower()
    return t
from typing import Sequence, Tuple, List, Optional
import torch
import numpy as np

def _normalize_token(t: str, marker: Optional[str] = None) -> str:
    markers = []
    if marker: markers.append(marker)
    markers += ['▁', 'Ġ', '##']
    for m in markers:
        t = t.replace(m, '')
    return t.lower()

def align_strict_one_to_one(
    base_vals: torch.Tensor,            
    blend_vals: torch.Tensor,           
    path: Sequence[Tuple[int, int]],    
    base_tokens: List[str],
    blend_tokens: List[str],
    base_marker: str,
    blend_marker: str,
    specTok_mapper: dict = None,
    *,
    debug: bool = False,
    max_print: int = 20,
    dtw_matrix: Optional[np.ndarray] = None,   # in lát cắt
    dtw_crop: int = 12,                         # kích thước lát cắt để in
) -> Tuple[torch.Tensor, torch.Tensor, List[Tuple[int,int]]]:
    if specTok_mapper is None:
        specTok_mapper = {}

    base_counts, blend_counts = {}, {}
    for i, j in path:
        base_counts[i] = base_counts.get(i, 0) + 1
        blend_counts[j] = blend_counts.get(j, 0) + 1

    base_norm  = [_normalize_token(t, base_marker)  for t in base_tokens]
    blend_norm = [_normalize_token(t, blend_marker) for t in blend_tokens]

    specTok_mapper_rev = {v: k for k, v in specTok_mapper.items()} if specTok_mapper else {}

    def _is_special_pair_ok(b_tok: str, s_tok: str) -> bool:
        if b_tok in specTok_mapper and specTok_mapper[b_tok] == s_tok:
            return True
        if s_tok in specTok_mapper_rev and specTok_mapper_rev[s_tok] == b_tok:
            return True
        return False

    one_to_one = [(i, j) for (i, j) in path
                  if base_counts.get(i, 0) == 1 and blend_counts.get(j, 0) == 1]

    kept_pairs, name_mismatch, multi_align = [], [], []
    for (i, j) in path:
        if base_counts.get(i, 0) != 1 or blend_counts.get(j, 0) != 1:
            if len(multi_align) < max_print:
                multi_align.append((i, j, base_tokens[i], blend_tokens[j],
                                    base_counts.get(i, 0), blend_counts.get(j, 0)))
            continue

        bi_raw, sj_raw = base_tokens[i], blend_tokens[j]
        if _is_special_pair_ok(bi_raw, sj_raw) or (base_norm[i] == blend_norm[j]):
            kept_pairs.append((i, j))
        else:
            if len(name_mismatch) < max_print:
                name_mismatch.append((i, j, bi_raw, sj_raw, base_norm[i], blend_norm[j]))

    if len(kept_pairs) == 0:
        A_base  = base_vals.new_empty((0, base_vals.size(-1)))
        A_blend = blend_vals.new_empty((0, blend_vals.size(-1)))
    else:
        A_base  = torch.stack([base_vals[i]  for (i, j) in kept_pairs], dim=0)
        A_blend = torch.stack([blend_vals[j] for (i, j) in kept_pairs], dim=0)

    if debug:
        print("\n================= [ALIGN DEBUG] =================")
        print(f"L_base={base_vals.size(0)}, L_blend={blend_vals.size(0)}, |path|={len(path)}")
        print(f"1–1 candidates: {len(one_to_one)} / {len(path)}")
        print(f"Final kept (strict name match + special map): {len(kept_pairs)}")

        if multi_align:
            print(f"\n[Examples dropped for multi-align] (show up to {max_print})")
            for (i, j, braw, sraw, bc, sc) in multi_align[:max_print]:
                print(f"  (i={i}, j={j}) teacher='{braw}' student='{sraw}'  counts=({bc},{sc})")

        if name_mismatch:
            print(f"\n[Examples dropped for name mismatch after normalize] (show up to {max_print})")
            for (i, j, braw, sraw, bn, sn) in name_mismatch[:max_print]:
                print(f"  (i={i}, j={j}) teacher='{braw}'→'{bn}'  vs  student='{sraw}'→'{sn}'")

        if len(kept_pairs) > 0:
            print(f"\n[First kept pairs] (up to {max_print}):")
            for (i, j) in kept_pairs[:max_print]:
                print(f"  (i={i}, j={j})  '{base_tokens[i]}' ↔ '{blend_tokens[j]}'  "
                      f"norm='{base_norm[i]}' ↔ '{blend_norm[j]}'")

        print(f"\nAligned 1–1 shapes: A_t={tuple(A_base.shape)}, A_s={tuple(A_blend.shape)}")

        if dtw_matrix is not None:
            H, W = dtw_matrix.shape
            h, w = min(dtw_crop, H), min(dtw_crop, W)
            print(f"\n[DTW matrix] shape={dtw_matrix.shape}  (show {h}x{w} top-left)")
            print(np.array2string(dtw_matrix[:h, :w], precision=2, suppress_small=True))
        print("=================================================\n")

    return A_base, A_blend

def align_by_path_pool_many(base_vals: torch.Tensor,
                            blend_vals: torch.Tensor,
                            path):
    A_base, A_blend = [], []
    k = 0
    P = len(path)
    while k < P:
        i0, j0 = path[k]
        if k == P - 1:
            A_base.append(base_vals[i0])
            A_blend.append(blend_vals[j0])
            break

        i1, j1 = path[k+1]
        di, dj = i1 - i0, j1 - j0

        if dj == 0 and di == 1:
            i_run = [i0]
            j_fix = j0
            kk = k + 1
            while kk < P and path[kk][1] == j_fix and path[kk][0] == i_run[-1] + 1:
                i_run.append(path[kk][0])
                kk += 1
            A_base.append(base_vals[i_run].mean(dim=0))
            A_blend.append(blend_vals[j_fix])
            k = kk
            continue

        if di == 0 and dj == 1:
            j_run = [j0]
            i_fix = i0
            kk = k + 1
            while kk < P and path[kk][0] == i_fix and path[kk][1] == j_run[-1] + 1:
                j_run.append(path[kk][1])
                kk += 1
            A_base.append(base_vals[i_fix])
            A_blend.append(blend_vals[j_run].mean(dim=0))
            k = kk
            continue

        # Diagonal / chuyển hướng lẻ: 1-1
        A_base.append(base_vals[i0])
        A_blend.append(blend_vals[j0])
        k += 1

    A_base  = torch.stack(A_base,  dim=0)
    A_blend = torch.stack(A_blend, dim=0)
    return A_base, A_blend

def batch_ids_to_tokens(ids_2d: torch.Tensor, tokenizer, skip_special=False):
    # ids_2d: [B, L]
    out = []
    for row in ids_2d.tolist():
        out.append(tokenizer.convert_ids_to_tokens(row, skip_special_tokens=skip_special))
    return out
def entropy_weight(logits, beta=2.0, eps=1e-12):
    logp = F.log_softmax(logits, dim=-1)
    p = logp.exp()
    H = -(p * logp).sum(dim=-1)            
    H_norm = H / math.log(logits.size(-1))  
    w = torch.exp(-beta * H_norm)           
    return w.detach()
print("Done Prepare Method")

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
from scipy.stats import pearsonr, spearmanr

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

class STSDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # instruction = "Given a text, Retrieve semantically similar text: "
        instruction=""
        return {
            "sentence1": instruction + self.dataset.iloc[idx]['sentence1'],
            "sentence2": instruction + self.dataset.iloc[idx]['sentence2'],
            "label": torch.tensor(self.dataset.iloc[idx]['score'], dtype=torch.float),
        }
        
def collate_fn(batch, tokenizer, max_len=128):
    s1_list = [item["sentence1"] for item in batch]
    s2_list = [item["sentence2"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])

    enc1 = tokenizer(
        s1_list,
        truncation=True,
        padding=True,       # chỉ pad theo câu dài nhất trong batch
        max_length=max_len,
        return_tensors="pt"
    )
    enc2 = tokenizer(
        s2_list,
        truncation=True,
        padding=True,
        max_length=max_len,
        return_tensors="pt"
    )

    return {
        "input_ids1": enc1["input_ids"],
        "attention_mask1": enc1["attention_mask"],
        "input_ids2": enc2["input_ids"],
        "attention_mask2": enc2["attention_mask"],
        "labels": labels,
    }

def eval_sts(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                input_ids2 = batch["input_ids2"].to(device)
                attn2 = batch["attention_mask2"].to(device)
                label = batch["labels"]


                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                out2 = model(input_ids=input_ids2, attention_mask=attn2)

                # emb1 = mean_pooling(out1, attn1)
                # emb2 = mean_pooling(out2, attn2)
                emb1 = out1.last_hidden_state[:, 0, :]
                emb2 = out2.last_hidden_state[:, 0, :]
                # emb1 = out1.last_hidden_state[:, -1, :]
                # emb2 = out2.last_hidden_state[:, -1, :]
        
                # cosine similarity
                sim = F.cosine_similarity(emb1, emb2)
                score = (sim + 1) * 2.5  # scale [-1,1] -> [0,5]
        
                preds.extend(score.cpu().numpy())
                labels.extend(label.numpy())
    
    spearman_corr, _ = spearmanr(preds, labels)
    print(f"Spearman: {spearman_corr:.4f}")

    return spearman_corr


def eval_sts_task(model, path_list):
    model.eval()
    print(' ✅ eval_sts_task')
    for path in path_list:
        print(path)
        eval_dataset = STSDataset(path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: collate_fn(x, tokenizer)
        )
        eval_sts(model, eval_loader)
    model.train()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
import datasets
import numpy as np
import torch

def eval_cls(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                label = batch["labels"]

                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                emb1 = out1.last_hidden_state[:, 0, :]
        
                preds.extend(emb1.cpu().numpy())
                labels.extend(label.numpy())
    
    return preds, labels

class ClasssifyDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return {
            "text": self.dataset.iloc[idx]['text'],
            "label": torch.tensor(self.dataset.iloc[idx]['label'], dtype=torch.long),
        }

def clf_collate_fn(batch, tokenizer, max_len=512):
    s1_list = [item["text"] for item in batch]
    labels = torch.stack([item["label"] for item in batch])

    enc1 = tokenizer(
        s1_list,
        truncation=True,
        padding=True,       # chỉ pad theo câu dài nhất trong batch
        max_length=max_len,
        return_tensors="pt"
    )

    return {
        "input_ids1": enc1["input_ids"],
        "attention_mask1": enc1["attention_mask"],
        "labels": labels,
    }


def eval_classification_task(model, path_list):
    model.eval()
    print(' ✅ eval classifier')

    for train_path, dev_path in path_list:
        print(dev_path)
        eval_dataset = ClasssifyDataset(dev_path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: clf_collate_fn(x, tokenizer)
        )
        
        train_dataset = ClasssifyDataset(train_path)
        train_loader = DataLoader(
            train_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: clf_collate_fn(x, tokenizer)
        )

        X_train, y_train = eval_cls(model, train_loader)
        X_test, y_test = eval_cls(model, eval_loader)

        clf = LogisticRegression(
            random_state=42,
            n_jobs=1,
            max_iter=200,
            verbose=0,
        )
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        scores = {}
        accuracy = accuracy_score(y_test, y_pred)
        scores["accuracy"] = accuracy
        f1 = f1_score(y_test, y_pred, average="macro")
        scores["f1"] = f1
        print(scores)
        
    model.train()

In [ ]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, average_precision_score


tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-uncased')

class PairDataset(Dataset):
    def __init__(self, file_path):
        self.dataset = pd.read_csv(file_path)

    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # instruction = "Given a text, Retrieve semantically similar text: "
        instruction=""
        return {
            "sentence1": instruction + self.dataset.iloc[idx]['sentence1'],
            "sentence2": instruction + self.dataset.iloc[idx]['sentence2'],
            "label": torch.tensor(self.dataset.iloc[idx]['label'], dtype=torch.float),
        }
        

def eval_pair(model, eval_loader):
    preds, labels = [], []
    device = model.device
    
    with torch.cuda.amp.autocast(dtype=torch.float16):
        with torch.no_grad():
            for batch in tqdm(eval_loader):
                input_ids1 = batch["input_ids1"].to(device)
                attn1 = batch["attention_mask1"].to(device)
                input_ids2 = batch["input_ids2"].to(device)
                attn2 = batch["attention_mask2"].to(device)
                label = batch["labels"]


                out1 = model(input_ids=input_ids1, attention_mask=attn1)
                out2 = model(input_ids=input_ids2, attention_mask=attn2)

                emb1 = out1.last_hidden_state[:, 0, :]
                emb2 = out2.last_hidden_state[:, 0, :]
                # emb1 = out1.last_hidden_state[:, -1, :]
                # emb2 = out2.last_hidden_state[:, -1, :]
        
                # cosine similarity
                sim = F.cosine_similarity(emb1, emb2)
                score = (sim + 1) / 2
        
                preds.extend(score.cpu().numpy())
                labels.extend(label.numpy())
    
    metric = get_metric_pair_classification(preds, labels)
    print(metric)

    return metric

def get_metric_pair_classification(scores, labels):
    best_acc, best_thr = 0, 0
    for thr in np.linspace(0, 1, 200):
        preds = (scores >= thr).astype(int)
        acc = accuracy_score(labels, preds)
        if acc > best_acc:
            best_acc, best_thr = acc, thr
    preds = (scores >= best_thr).astype(int)
    return {
        "best_threshold": best_thr,
        "accuracy": best_acc,
        "f1": f1_score(labels, preds, average="macro"),
        "precision": precision_score(labels, preds, average="macro"),
        "recall": recall_score(labels, preds, average="macro"),
        "average_precision": average_precision_score(labels, scores)
    }


def eval_pair_task(model, path_list):
    model.eval()
    print(' ✅ eval_pair_task')
    for path in path_list:
        print(path)
        eval_dataset = PairDataset(path)
        eval_loader = DataLoader(
            eval_dataset,
            batch_size=64,
            shuffle=False,
            collate_fn=lambda x: collate_fn(x, tokenizer)
        )
        eval_pair(model, eval_loader)
    model.train()

In [ ]:
eval_cls_tasks = [('/kaggle/input/multitask-data/multi-data/banking_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/banking77_validation.csv'),
                  ('/kaggle/input/multitask-data/multi-data/emotion_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/emotion_validation.csv'), 
                  ('/kaggle/input/multitask-data/multi-data/tweet_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/tweet_validation.csv')]

eval_sts_tasks = ['/kaggle/input/multitask-data/multi-data/sick_validation.csv', 
                  '/kaggle/input/multitask-data/multi-data/sts12_validation.csv', 
                  '/kaggle/input/multitask-data/multi-data/stsb_validation.csv']

eval_pair_tasks = ['/kaggle/input/multitask-data/multi-data/mrpc_validation.csv', 
                   '/kaggle/input/multitask-data/multi-data/scitail_validation.csv', 
                   '/kaggle/input/multitask-data/multi-data/wic_validation.csv']
test_cls_tasks = [('/kaggle/input/multitask-data/multi-data/banking_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/banking77_test.csv'),
                  ('/kaggle/input/multitask-data/multi-data/emotion_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/emotion_test.csv'), 
                  ('/kaggle/input/multitask-data/multi-data/tweet_train.csv', 
                   '/kaggle/input/multitask-data/multi-data/tweet_test.csv')]

test_sts_tasks = ['/kaggle/input/multitask-data/multi-data/sick_test.csv', 
                  '/kaggle/input/multitask-data/multi-data/sts12_test.csv', 
                  '/kaggle/input/multitask-data/multi-data/stsb_test.csv']

test_pair_tasks = ['/kaggle/input/multitask-data/multi-data/mrpc_test.csv', 
                   '/kaggle/input/multitask-data/multi-data/scitail_test.csv', 
                   '/kaggle/input/multitask-data/multi-data/wic_test.csv']

# model_student.to("cuda")
# eval_classification_task(model_student, test_cls_tasks)
# eval_pair_task(model_student, test_pair_tasks)
# eval_sts_task(model_student, test_sts_tasks)

In [ ]:
import torch
import torch.nn.functional as F

def _minmax_rescale(x: torch.Tensor, eps: float = 1e-8, lo: float = 0.5, hi: float = 1.0):
    # chuẩn hoá min-max rồi map về [lo, hi]
    x = x - x.min()
    x = x / (x.max() + eps)
    return lo + (hi - lo) * x

def _rescale_scalar(p: float, lo: float = 0.85, hi: float = 1.15):
    return lo + (hi - lo) * float(p)

def make_series_factors_from_inbatch_logits(
    logits_s: torch.Tensor, logits_t: torch.Tensor,
    i: int,
    Ti: torch.Tensor, Si: torch.Tensor,
    mode: str = "token_weighted",   # "token_weighted" | "broadcast"
    eps: float = 1e-8
):
    with torch.no_grad():
        Pt_row = F.softmax(logits_t[i], dim=-1).clamp_min(eps) 
        Ps_row = F.softmax(logits_s[i], dim=-1).clamp_min(eps)  

        # độ tự tin positive (đường chéo)
        conf_t = Pt_row[i].item()
        conf_s = Ps_row[i].item()

        ft = _rescale_scalar(conf_t, lo=0.85, hi=1.15) 
        fs = _rescale_scalar(conf_s, lo=0.85, hi=1.15)  

        Lt = Ti.size(0)
        Ls = Si.size(0)

        if mode == "broadcast":
            f1 = torch.full((Lt,), ft, dtype=torch.float32, device=Ti.device)
            f2 = torch.full((Ls,), fs, dtype=torch.float32, device=Si.device)
        else:
            w_tok_t = _minmax_rescale(Ti.norm(dim=-1).detach().float(), lo=0.6, hi=1.0)
            w_tok_s = _minmax_rescale(Si.norm(dim=-1).detach().float(), lo=0.6, hi=1.0)
            f1 = (w_tok_t * ft).clamp(0.5, 2.0)
            f2 = (w_tok_s * fs).clamp(0.5, 2.0)

    return f1, f2


In [ ]:
best_f1_macro = -1.0
all_preds, all_labels = [], []
total_loss = 0.0

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


num_steps = len(train_loader)
total_traning_steps = num_steps * epochs
warmup_ratio = 0.1

def pick_devices():
    if torch.cuda.device_count() >= 2:
        dev_s = torch.device("cuda:0")  # student
        dev_t = torch.device("cuda:1")  # teacher
    elif torch.cuda.is_available():
        print("[WARN] Only 1 GPU available -> both on cuda:0")
        dev_s = dev_t = torch.device("cuda:0")
    else:
        print("[WARN] No GPU -> CPU")
        dev_s = dev_t = torch.device("cpu")
    return dev_s, dev_t

device_s, device_t = pick_devices()

model_student.to(device_s)
model_teacher.to(device_t)
model_teacher.eval()
for p in model_teacher.parameters():
    p.requires_grad_(False)

proj_s2t = None

optimizer = optim.AdamW(model_student.parameters(), lr=LR)
scaler = GradScaler(enabled=torch.cuda.is_available())
scheduler = get_scheduler(
    name='cosine_with_min_lr',
    optimizer=optimizer,
    num_warmup_steps=int(total_traning_steps * warmup_ratio),
    # num_warmup_steps=1,
    num_training_steps=total_traning_steps,
    scheduler_specific_kwargs={'min_lr': 2e-6}
)
if save_dir:
    os.makedirs(save_dir, exist_ok=True)

n_items = 0
use_task_loss = True

In [ ]:
w_task     = 0.5       # bi encoder
alpha_dtw  = 0.5       # KD_DTW
beta_score = 0.0       # mse matmul
DTW_FACTOR_MODE = "token_weighted"
for epoch in range(epochs):
    model_student.train()
    total_loss, n_items = 0.0, 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

    for batch in pbar:
        # ---- move to device_s / device_t ----
        batch_s, batch_t = {}, {}
        for k, v in batch.items():
            if not torch.is_tensor(v):
                continue
            if k.endswith("_stu") or k == "labels":
                batch_s[k] = v.to(device_s, non_blocking=True)
            if k.endswith("_tea"):
                batch_t[k] = v.to(device_t, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        with autocast(enabled=torch.cuda.is_available()):
            # ========== TEACHER (bi-encoder) trên device_t ==========
            with torch.inference_mode():
                t_out1 = model_teacher(
                    input_ids=batch_t["input_ids1_tea"],
                    attention_mask=batch_t["attention_mask1_tea"],
                    output_hidden_states=True, return_dict=True
                )
                t_out2 = model_teacher(
                    input_ids=batch_t["input_ids2_tea"],
                    attention_mask=batch_t["attention_mask2_tea"],
                    output_hidden_states=True, return_dict=True
                )
                T_last1 = t_out1.hidden_states[-1]   # [B, L1, d_t] (device_t)
                T_last2 = t_out2.hidden_states[-1]   # [B, L2, d_t]
                T_cls1  = T_last1[:, 0, :]           # [B, d_t]
                T_cls2  = T_last2[:, 0, :]

                T_last1 = T_last1.to(device_s, non_blocking=True)
                T_last2 = T_last2.to(device_s, non_blocking=True)
                T_cls1  = T_cls1.to(device_s, non_blocking=True)
                T_cls2  = T_cls2.to(device_s, non_blocking=True)

            # ========== STUDENT (bi-encoder) trên device_s ==========
            s_out1 = model_student(
                input_ids=batch_s["input_ids1_stu"],
                attention_mask=batch_s["attention_mask1_stu"],
                output_hidden_states=True, return_dict=True
            )
            s_out2 = model_student(
                input_ids=batch_s["input_ids2_stu"],
                attention_mask=batch_s["attention_mask2_stu"],
                output_hidden_states=True, return_dict=True
            )
            S_last1 = s_out1.hidden_states[-1]       # [B, L1, d_s]
            S_last2 = s_out2.hidden_states[-1]       # [B, L2, d_s]
            S_cls1  = S_last1[:, 0, :]               # [B, d_s]
            S_cls2  = S_last2[:, 0, :]

            if proj_s2t is None:
                d_s, d_t = S_cls1.size(-1), T_cls1.size(-1)
                proj_s2t = torch.nn.Linear(d_s, d_t, bias=False).to(device_s)
                optimizer.add_param_group({"params": proj_s2t.parameters(), "lr": LR})

            S_proj_cls1 = proj_s2t(S_cls1)          # [B, d_t]
            S_proj_cls2 = proj_s2t(S_cls2)          # [B, d_t]

            # dtype chuẩn theo autocast
            base_dtype = S_last1.dtype
            zero = torch.zeros((), device=device_s, dtype=base_dtype)

            # ========== (A) HARD TASK LOSS: bi-encoder STS ==========
            
            loss_task, logit_student_inbatch = info_nce(S_cls1, S_cls2)
            _,        logit_teacher_inbatch = info_nce(T_cls1, T_cls2)


            # ========== (B) KD_DTW: token-level cho từng câu ==========
            kd_sum, denom = 0.0, 0
            keep_s1 = batch_s["attention_mask1_stu"].bool() & (~batch_s["special_tokens_mask1_stu"].bool())
            keep_t1 = (batch["attention_mask1_tea"].to(device_s).bool()
                       & (~batch["special_tokens_mask1_tea"].to(device_s).bool()))
            
            Bsz = S_last1.size(0)
            for i in range(Bsz):
                # ===== câu 1 =====
                stu_tok_full = tok_student.convert_ids_to_tokens(
                    batch["input_ids1_stu"][i].cpu().tolist(), skip_special_tokens=False
                )
                tea_tok_full = tok_teacher.convert_ids_to_tokens(
                    batch["input_ids1_tea"][i].cpu().tolist(), skip_special_tokens=False
                )
                s_tok_i = [t for t, m in zip(stu_tok_full, keep_s1[i].detach().cpu().tolist()) if m]
                t_tok_i = [t for t, m in zip(tea_tok_full, keep_t1[i].detach().cpu().tolist()) if m]
            
                Si = S_last1[i][keep_s1[i]]   # [Ns1_i, d_s]
                Ti = T_last1[i][keep_t1[i]]   # [Nt1_i, d_t]

                f1, f2 = make_series_factors_from_inbatch_logits(
                    logits_s=logit_student_inbatch.detach(),   
                    logits_t=logit_teacher_inbatch.detach(),   
                    i=i,
                    Ti=Ti,  
                    Si=Si, 
                    mode=DTW_FACTOR_MODE
                )
                
                series1_factor = f1.detach().cpu().numpy()   
                series2_factor = f2.detach().cpu().numpy()  

                if Si.numel() > 0 and Ti.numel() > 0 and len(s_tok_i) > 0 and len(t_tok_i) > 0:
                    matches, *_ = dtw(series_1=t_tok_i, series_2=s_tok_i,series1_factor=series1_factor,series2_factor=series2_factor, norm_func=dist_fn)
            
                    Nt, Ns = Ti.size(0), Si.size(0)
            
                    A_t, A_s = align_by_path_pool_many(
                        base_vals=Ti, blend_vals=Si,
                        # base_tokens=t_tok_i, blend_tokens=s_tok_i,
                        # base_marker=base_model_special_token, blend_marker=blending_model_special_token,
                        # specTok_mapper=specTok_mapper,
                        path=matches
                    )
                    S_proj_tok = proj_s2t(A_s).to(base_dtype)
                    A_t = A_t.to(base_dtype)
                    kd_sum += F.mse_loss(S_proj_tok, A_t, reduction="sum")
                    denom  += A_t.numel()
                    del S_proj_tok, A_t, A_s
            


            kd_dtw = torch.tensor(0.0, device=device_s, dtype=base_dtype) if denom == 0 else \
                     (kd_sum / denom).to(device=device_s, dtype=base_dtype)

            # ========== (C) KD_SCORE: pairwise cosine matrix ==========
            t_norm = F.normalize(T_cls1, dim=-1).to(base_dtype)
            s_norm = F.normalize(S_proj_cls1, dim=-1)    
            teacher_scores = t_norm @ t_norm.transpose(0, 1)  
            student_scores = s_norm @ s_norm.transpose(0, 1)   
            kd_score = F.mse_loss(student_scores, teacher_scores)

            # ========== TỔNG LOSS ==========
            loss = w_task * loss_task + alpha_dtw * (kd_dtw + beta_score * kd_score)
            loss = loss.float()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        # ----- logging -----
        bs = next(iter(batch_s.values())).size(0) 
        total_loss += loss.item() * bs
        n_items    += bs

        mem_info = {}
        for dev_id in range(torch.cuda.device_count()):
            mem_alloc = torch.cuda.memory_allocated(dev_id) / 1024**2
            mem_reserved = torch.cuda.memory_reserved(dev_id) / 1024**2
            mem_info[f"gpu{dev_id}"] = f"{mem_alloc:.0f}/{mem_reserved:.0f}MB"

        avg_loss = total_loss / max(1, n_items)
        pbar.set_postfix({"avg_loss": f"{avg_loss:.4f}", **mem_info})

        # dọn bộ nhớ
        del S_last1, S_last2, T_last1, T_last2, S_cls1, S_cls2, T_cls1, T_cls2, loss
        torch.cuda.empty_cache()

    eval_classification_task(model_student, test_cls_tasks)
    eval_pair_task(model_student, test_pair_tasks)
    eval_sts_task(model_student, test_sts_tasks)
